<h1 align="center">Software Introspection for Signaling Emergent Cyber-Social Operations (SIGNAL)</h1>
<h2 align="center">SRI International</h2>
<h3 align="center">In support of DARPA AIE Hybrid AI to Protect Integrity of Open Source Code (SocialCyber)</h3>

## Introduction

In this notebook, we provide pretrained RandomForest models trained on the Persuasion for Good (P4G) dataset from Wang et al. [1]. For the interested reader, the P4G dataset is publicly available on GitLab, see [HERE](https://gitlab.com/ucdavisnlp/persuasionforgood/tree/master/data). Wang et al. emphasize the need for understanding the intrinsic disclosure and appeal strategies taking place in human persuasion conversations. In their words, being able to model such properties would lead to advancements in the ethical developments of automated dialogue systems [1].

In the context of the SIGNAL project, and SocialCyber overall program, we foresaw three main benefits resulting from the adoption and use of P4G dataset, namely:
1. Assist in enriching the developer communication networks.,
2. Help in understanding and modeling the developer communications in the LKML.,
3. Provide an additional axis of information, indicating the persuasion strategies used by developers when trying to push their patches upstream in the Linux Kernel.

### In this tutorial...

We will start by demonstrating the performance of a Random Forest ([RF](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)) model pre-trained on P4G data. We opted for Random Forests, as the original Hybrid-RCNN architecture presented by [1] would take a substantial amount of time to train on the data (When testing Hybrid-RCNN, one single round of training required more than 12-hours when run over a single GPU). Furthermore, RFs provided comparable accuracy to the Hybrid-RCNN.

#### Transfer Learning

To adapt the RF model to the LKML data, we made use of *Transfer Learning* techniques. Transfer Learning enables re-purposing *trained* machine learning (ML) models to new tasks, as long as these tasks share some similarities with one another. TL is heavily used in deep learning tasks, where data or resource contraints prohibit the research community to train DL models from scratch.

##### Structure Expansion Reduction Algorithm

To enable the use of transfer learning on RF models, we turned our attention to the work of Segev et al. [2], where the authors introduce the structure expansion reduction (**SER**) algorithm. SER has two variants: 1) The first variant greedily searches and applies local modifications to each decision tree, expanding or reducing the tree around individual nodes; 2) The second variant of SER works by modifying the parameters of a selected decision tree component. We implement both of these variants using the original work by Segev et al. [2] and the subsequent work by Minvielle et al. [3] as basis.

#### References

[1] Wang, Xuewei, Weiyan Shi, Richard Kim, Yoojung Oh, Sijia Yang, Jingwen Zhang, and Zhou Yu. "*Persuasion for good: Towards a personalized persuasive dialogue system for social good.*" Proceedings of the 57th Annual Meeting of the Association for Computational Linguistics. ACL (2019). [PDF](https://arxiv.org/pdf/1906.06725.pdf).

[2] Segev, Noam, Maayan Harel, Shie Mannor, Koby Crammer, and Ran El-Yaniv. "*Learn on source, refine on target: A model transfer learning framework with random forests.*" IEEE transactions on pattern analysis and machine intelligence 39, no. 9 (2016): 1811-1824. [PDF](https://arxiv.org/pdf/1511.01258.pdf).

[3] Minvielle, Ludovic, Mounir Atiq, Sergio Peignier, and Mathilde Mougeot. "*Transfer Learning on Decision Tree with Class Imbalance.*" In 2019 IEEE 31st International Conference on Tools with Artificial Intelligence (ICTAI), pp. 1003-1010. IEEE, 2019. [PDF](https://ieeexplore.ieee.org/abstract/document/8995296).

#### Disclaimer

The content of this notebook is released under the **GNU General Public License v3.0**, see [LICENSE](https://github.com/SRI-CSL/signal-public/blob/main/LICENSE).

## Demo

In [8]:
import sys
import os

import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

from ser.ser_trees import * 
import ser.api as api

ModuleNotFoundError: No module named 'ser_algo'

In [4]:
PARENT_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(PARENT_DIR)

DATASETS_DIR = os.path.join(PARENT_DIR, 'data')
MODELS_DIR = os.path.join(PARENT_DIR, 'models')

In [5]:
id2label = {0: 'task-related-inquiry',
             1: 'credibility-appeal',
             2: 'logical-appeal',
             3: 'personal-related-inquiry',
             4: 'source-related-inquiry',
             5: 'donation-information',
             6: 'foot-in-the-door',
             7: 'emotion-appeal',
             8: 'self-modeling',
             9: 'personal-story'}

In [6]:
tfidf_file_path = os.path.join(MODELS_DIR, 'tfidf_09172021.pkl')
rf_base_file_path = os.path.join(MODELS_DIR, 'rfclf_09172021.pkl')
rf_ser_file_path = os.path.join(MODELS_DIR, 'rf_tl_model_red.pkl')

with open(tfidf_file_path, 'rb') as tf:
    tfidf = pickle.load(tf)

with open(rf_base_file_path, 'rb') as rb:
    rf_base = pickle.load(rb)

# with open(rf_ser_file_path, 'rb') as rs:
#     rf_ser = pickle.load(rs)

/anaconda2/envs/signal-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/anaconda2/envs/signal-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/anaconda2/envs/signal-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 1.0.2. This m

In [7]:
id2label[rf_base.predict(tfidf.transform(["Are you involved with charities?"]))[0]]

'task-related-inquiry'